In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import re
import json
from pymongo import MongoClient
from odo import odo
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
with open('art_links') as f:
    data = f.readlines()
    data = [data.rstrip('\n') for data in open('art_links')]

data = filter(None, data)
data = data[12000:]

In [ ]:
soup_objs = []

for line in data:
    response = requests.get(line)
    page = response.text
    link_soup = BeautifulSoup(page)
    soup_objs.append(link_soup)
    
response.status_code

In [ ]:
from collections import defaultdict

client = MongoClient()
db = client.paintings

artwork_test = db.artwork_test

In [ ]:
for soup in soup_objs: 
    art_dict = defaultdict(list)
    dets = soup.findAll('dt', class_='collection-details__tombstone--label')
    for s in dets:
        art_dict[s.text].append(s.nextSibling.nextSibling.text)
    art_dict['title'] = soup.find('h1', class_='collection-details__object-title').encode('utf-8')
    #art_dict['artist'] = soup.find('div',class_='collection-details__tombstone').encode('utf-8')
    art_dict['gallery'] = soup.find('div', class_='collection-details__location').encode('utf-8')
    art_dict['description'] = soup.find('div', class_='collection-details__label').encode('utf-8')
    images = soup.findAll('img')
    for i in images:
        if 'ng-src' in str(i):
            art_dict['image']=i.encode('utf-8')
    artwork_test.insert_one(art_dict)

In [ ]:
artwork_test.count()

In [ ]:
type(artwork_test)

In [ ]:
tmp = pd.DataFrame({'artist':[1,2,3, 4], 'genre':[4,5,6, 5]})
tmp[tmp['artist'] == 2]['genre']
art= list(tmp[tmp['genre'] == 5]['artist'])

In [ ]:
tmp[tmp['artist'] == 2]['genre']

In [ ]:
art= list(tmp[tmp['genre'] == 5]['artist'])

In [ ]:
print list(artwork_test.find())[0].keys()
print list(artwork_test.find())[1].keys()

In [ ]:
import pandas as pd

df = pd.DataFrame(list(artwork_test.find()))

In [ ]:
df.head()

In [ ]:
import numpy as np

df['description'] = df['description'].str.replace('<div class="collection-details__label">\n</div>', '')
df['description'] = df['description'].str.replace('<div class="collection-details__label">\r\n', '')
df['description'] = df['description'].str.replace('<br/><br/>', '')
df['description'] = df['description'].str.replace('\r\n    </div>', '')
df['gallery'] = df['gallery'].str.replace('<div class="collection-details__location">\n<svg class="icon">\n<use xlink:href="/dist/icons.svg#ico-marker" xmlns:xlink="http://www.w3.org/1999/xlink"></use>\n</svg>\r\n', '')
df['gallery'] = df['gallery'].str.replace('\r\n    </div>', '')
df['title'] = df['title'].str.replace('<h1 class="collection-details__object-title">', '')
df['title'] = df['title'].str.replace('</h1>', '')
df['title'] = df['title'].str.replace('\r\n<br/>', '')
df['title'] = df['title'].str.replace('<br/>', '')
#df['gallery'] = df['gallery'].str.replace('        On view at The Met Fifth Avenue in Gallery ', '')

In [ ]:
len(df)

In [ ]:
df.head(2)

In [ ]:
df = df.drop_duplicates('_id')

In [ ]:
len(df)

In [ ]:
list(df.columns.values)

In [ ]:
df['Date:'].isnull().sum()

In [ ]:
df['Date:'][:10]

In [ ]:
df['Geography:'].isnull().sum()

In [ ]:
df['Culture:'].isnull().sum()

In [ ]:
df['Period:'].isnull().sum()

In [ ]:
df.to_csv('my_df_1.csv', sep='\t', encoding='utf-8')

In [ ]:
import nltk
nltk.download('maxent_treebank_pos_tagger')

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('words')

In [ ]:
from nltk.tokenize import sent_tokenize
sentences = [sent_tokenize(x.lower()) for x in df['description']]
print sentences[0]

In [ ]:
from nltk.tokenize import word_tokenize

words = [word_tokenize(x) for x in df['description']]
print words[0]

In [ ]:
nltk.download()

In [ ]:
from nltk.tag import pos_tag
word_tags = word_tokenize(df['description'].str.lower()[0])
pos_tag(word_tags)

In [ ]:
!pip install textblob

In [ ]:
from textblob import TextBlob

In [ ]:
blob = [TextBlob(x) for x in (df['description'])]
blob[10]

In [ ]:
blob[10].sentiment

In [ ]:
stemmer = nltk.stem.porter.PorterStemmer()

for word in blob[0].words:
    print stemmer.stem(word)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df_lower = [i.lower() for i in df['description']]

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
count_vect = vectorizer.fit_transform([i for i in df['description']])
#count_vect = vectorizer.fit_transform(df_lower)
count_vect

In [ ]:
count_vect[:100].toarray()

In [ ]:
bigram_vectorizer = CountVectorizer(stop_words= 'english', ngram_range=(2,2),min_df=1)
bigram_vectorizer.fit_transform(df_lower[:100]).toarray()

In [ ]:
bigram_vectorizer.get_feature_names()[200:210]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit_transform([i for i in df_lower[:100]]).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

#text = ['hi mike', 'hi hi hi mike mike mike']
#X = vectorizer.fit_transform(text)

#print cosine_distances(X[0], X[1])

#print cosine_distances()
print cosine_distances(count_vect[2000], count_vect[80])

In [ ]:
count_vect[50]

In [ ]:
df_description = df.drop_duplicates('description')
df_description = df_description[['title', 'description']]
df_description.head(5)

In [ ]:
len(df_description)

In [ ]:
title = [i for i in df_description['title']]
print title[:5]

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
desc_count_vect = vectorizer.fit_transform([i for i in df_description['description']])

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
cosine_distances(desc_count_vect)
arr = cosine_distances(desc_count_vect)

In [ ]:
type(arr)

In [ ]:
cos_dist = pd.DataFrame(arr.T, columns=title)
cos_dist.head(50)

In [ ]:
se = pd.Series(title)
cos_dist['title'] = se.values
cos_dist.head()

In [ ]:
cos_dist = cos_dist.set_index('title')

In [ ]:
#similar
cos_dist.ix["The Dance Class", "The Dancing Class"]

In [ ]:
#different
cos_dist.ix['Seated Harlequin', 'Young Girl Bathing']

In [ ]:
#different
cos_dist.ix['Charles Suisse', 'Before and After I']

In [ ]:
#similar
cos_dist.ix['No. 3', 'No. 21']

In [ ]:
vectorizer_nostop = CountVectorizer()
desc_count_vect_nostop = vectorizer_nostop.fit_transform([i for i in df_description['description']])
cosine_distances(desc_count_vect_nostop)

In [ ]:
arr_nostop = cosine_distances(desc_count_vect_nostop)
cos_dist_nostop = pd.DataFrame(arr_nostop.T, columns=title)
se = pd.Series(title)
cos_dist_nostop['title'] = se.values
cos_dist_nostop.head()

In [ ]:
cos_dist_nostop = cos_dist_nostop.set_index('title')

In [ ]:
#similar
cos_dist_nostop.ix["The Dance Class", "The Dancing Class"]

In [ ]:
#different
cos_dist_nostop.ix['Seated Harlequin', 'Young Girl Bathing']

In [ ]:
#different
cos_dist_nostop.ix['Charles Suisse', 'Before and After I']

In [ ]:
#similar
cos_dist_nostop.ix['No. 3', 'No. 21']

In [ ]:
tfidf_vect = TfidfVectorizer(stop_words='english')
tfidf_vect = tfidf_vect.fit_transform([i for i in df_description['description']])
tfidf_dist = cosine_distances(tfidf_vect)

In [ ]:
arr_tfidf = tfidf_dist
cos_dist_tfidf = pd.DataFrame(arr_tfidf.T, columns=title)
se = pd.Series(title)
cos_dist_tfidf['title'] = se.values

In [ ]:
cos_dist_tfidf = cos_dist_tfidf.set_index('title')
cos_dist_tfidf.head()

In [ ]:
#similar
cos_dist_tfidf.ix["The Dance Class", "The Dancing Class"]

In [ ]:
#different
cos_dist_tfidf.ix['Seated Harlequin', 'Young Girl Bathing']

In [ ]:
#different
cos_dist_tfidf.ix['Charles Suisse', 'Before and After I']

In [ ]:
#similar
cos_dist_tfidf.ix['No. 3', 'No. 21']

In [ ]:
len(cos_dist_tfidf)

In [ ]:
artists = pd.read_csv('artists.csv')
artists['Unnamed: 1'].replace(np.nan, '?', inplace=True)

In [ ]:
artists = pd.read_csv('artists.csv')
artists['Unnamed: 1'].replace(np.nan, '?', inplace=True)

In [ ]:
artists.columns

In [ ]:
artist_list = artists['Unnamed: 1'].tolist()

In [ ]:
strings = ';'.join(artist_list)

In [ ]:
strings.split('?')

In [ ]:
split_cats = [category.split(';') for category in strings.split('?')]
split_cats = [ [e for e in cat if e] for cat in split_cats]
split_cats = [c for c in split_cats if c]
split_cats

In [ ]:
artist_dict = {cat[0].replace(':',''): cat[1:] for cat in split_cats}

In [ ]:
split_cats[1][0]

In [ ]:
len(artist_dict)

In [ ]:
artist_dict.keys()

In [ ]:
movement_artist = pd.DataFrame(artist_dict.items())
movement_artist.columns = ['movement', 'artist_move']
movement_artist.head()

In [ ]:
move_art = movement_artist.groupby('movement').artist_move.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

In [ ]:
move_art.columns = ['movement', 'artist_name']
move_art.head()

In [ ]:
art = list(tmp[tmp['genre'] == 5]['artist'])

In [ ]:
move_art[move_art['movement'] == 'Taisho Period']

In [ ]:
painting_lookup_temp = dict(df_description['title'])
len(painting_lookup_temp)

In [ ]:
painting_lookup = {}
for k, v in painting_lookup_temp.items():
    painting_lookup[v] = k

painting_lookup

In [ ]:
import operator

def recommended_paintings(painting):
    new_df = cos_dist_tfidf[painting]
    #print new_df
    tups = new_df.iteritems()
    tups = list(tups)
    tups.sort(key=lambda x: x[1])
    return tups[1:6]

In [ ]:
recommended_paintings('Seated Four-Armed Ganesha')

In [ ]:
recommended_paintings('Still Life: Flowers and Fruit')

In [ ]:
recommended_paintings('The Businessman Max Roesberg, Dresden')

In [ ]:
recommended_paintings('The Dance Class')

In [ ]:
recommended_paintings('Washington Crossing the Delaware')

In [ ]:
recommended_paintings('Portia')

In [ ]:
def gallery_number(painting):
    pd.options.display.max_colwidth = 100
    row = df.loc[df['title'] == painting]
    gallery = row['gallery']
    if gallery.str.contains('Not on view').any():
        return None
    else:
        return str(gallery)

In [ ]:
gallery_number('Seated Four-Armed Ganesha')

In [ ]:
gallery_number('Mademoiselle Marie Dihau')

In [ ]:
medieval = open('medieval.txt').read().splitlines()
proto_renaissance = open('proto_renaissance.txt').read().splitlines()
early_renaissance = open('early_renaissance.txt').read().splitlines()
high_renaissance = open('high_renaissance.txt').read().splitlines()
mannerism = open('mannerism.txt').read().splitlines()
northern_renaissance = open('northern_renaissance.txt').read().splitlines()
baroque = open('baroque.txt').read().splitlines()
rococo = open('rococo.txt').read().splitlines()
neoclassical = open('neoclassical.txt').read().splitlines()
academic_art = open('academic_art.txt').read().splitlines()
romanticism = open('romanticism.txt').read().splitlines()
realism = open('realism.txt').read().splitlines()
naturalism = open('naturalism.txt').read().splitlines()
naive_primitivism = open('naive_primitivism.txt').read().splitlines()
symbolism = open('symbolism.txt').read().splitlines()
aestheticism = open('aestheticism.txt').read().splitlines()
tonalism = open('tonalism.txt').read().splitlines()
impressionism = open('impressionism.txt').read().splitlines()
post_impressionism = open('post_impressionism.txt').read().splitlines()
art_neauvau = open('art_neauvau.txt').read().splitlines()
pictoralism = open('pictoralism.txt').read().splitlines()
expressionism = open('expressionism.txt').read().splitlines()
neo_romananticism = open('new_romanticism.txt').read().splitlines()
cubism = open('cubism.txt').read().splitlines()
abstract_art = open('abstract_art.txt').read().splitlines()
futurism = open('futurism.txt').read().splitlines()
dada = open('dada.txt').read().splitlines()
constructivism = open('constructivism.txt').read().splitlines()
modernismo = open('modernismo.txt').read().splitlines()
concretism = open('concretism.txt').read().splitlines()
social_realism = open('social_realism.txt').read().splitlines()
surrealism = open('surrealism.txt').read().splitlines()
magic_realism = open('magic_realism.txt').read().splitlines()
art_deco = open('art_deco.txt').read().splitlines()
abstract_expressionism = open('abstract_expressionism.txt').read().splitlines()
art_informel = open('art_informel.txt').read().splitlines()
tachisme = open('tachisme.txt').read().splitlines()
post_painterly_abstraction = open('post_painterly_abstraction.txt').read().splitlines()
feminist_art = open('feminist_art.txt').read().splitlines()
outsider_art = open('outsider_art.txt').read().splitlines()
neo_expressionism = open('neo_expressionism.txt').read().splitlines()
neo_dada = open('neo_dada.txt').read().splitlines()
kinetic_art = open('kinetic_art.txt').read().splitlines()
arte_povera = open('arte_povera.txt').read().splitlines()
op_art = open('op_art.txt').read().splitlines()
pop_art = open('pop_art.txt').read().splitlines()
contemporary = open('contemporary.txt').read().splitlines()
minimalism = open('minimalism.txt').read().splitlines()
chinese_art = open('chinese_art.txt').read().splitlines()
edo_period = open('edo_period.txt').read().splitlines()
meiji_period = open('meiji_period.txt').read().splitlines()
taisho_period = open('taisho_period.txt').read().splitlines()
showa_period = open('showa_period.txt').read().splitlines()
islamic_art = open('islamic_art.txt').read().splitlines()

In [ ]:
def handle_row(x):
    if x=='nan':
        return ''
    else:
        try:
            tmp = ''.join(eval(x))
        except:
            print x
            return ''
        return tmp
handle_row(df['artist_name'].values[0])

df['artist_name'] = map(handle_row, df['artist_name'].values)

In [ ]:
df_movement = pd.merge(df, move_art, on='artist_name', how='inner')
df_movement = df_movement.drop_duplicates('description')

In [ ]:
df_movement[100:150]

In [ ]:
df_movement[df_movement['movement'] == 'Impressionism'][200:]

In [ ]:
len(df_movement)

In [ ]:
def filter_df(painting):
    try:
        row = df_movement.loc[df_movement['title'] == painting]
        move = row['movement']
        df_select = df_movement[df_movement.movement.isin(move)]
        df_select = cos_dist_tfidf[painting]
        tups = df_select.iteritems()
        tups = list(tups)
        tups.sort(key=lambda x: x[1])
        return tups[1:6]
    except:
        recommended_paintings(painting)

In [ ]:
filter_df('Mr. and Mrs. I. N. Phelps Stokes')